In [8]:
from zipfile import ZipFile
import numpy as np
import pandas as pd
import os
from StringIO import StringIO

In [11]:
def load_dataset():
    def load_images(filename):
        if (os.path.exists(filename)):
            print filename
            # print ZipFile(filename, 'r').namelist()
            zf = ZipFile(filename, 'r')
            print len(zf.namelist())
            # TODO: put range len(zf.namelist())
            for i in range(1, 10):
                data = StringIO(zf.read(zf.namelist()[i]))
                print data
            
                
            return []

    def load_labels(filename):
        if (os.path.exists(filename)):
            print filename
            return pd.read_csv(ZipFile(filename, 'r').open('labels.csv'))

    X_train = load_images('data/train.zip')
    Y_train = load_labels('data/labels.csv.zip')
    X_test = []
    # X_test = load_images('data/test.zip')

    return X_train, Y_train, X_test

In [12]:
X_train_orig, Y_train_orig, X_test_orig = load_dataset()
# print X_train
# print Y_train

data/train.zip
10223
<StringIO.StringIO instance at 0x10e4abc20>
<StringIO.StringIO instance at 0x10e4abb90>
<StringIO.StringIO instance at 0x10e4abc20>
<StringIO.StringIO instance at 0x10e4abb90>
<StringIO.StringIO instance at 0x10e4abc20>
<StringIO.StringIO instance at 0x10e4abb90>
<StringIO.StringIO instance at 0x10e4abc20>
<StringIO.StringIO instance at 0x10e4abb90>
<StringIO.StringIO instance at 0x10e4abc20>
data/labels.csv.zip
